## Inderes login & password

In [1]:
import json 
# Edit secrets.json file with your login details
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    LOGIN_EMAIL = secrets['INDERES_LOGIN_EMAIL']
    LOGIN_PASSWORD = secrets['INDERES_LOGIN_PASSWORD']

## Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import time
import csv
import itertools
from collections import deque

In [3]:
import urllib.request
from urllib import robotparser
from urllib.parse import urljoin
from urllib.error import URLError, HTTPError, ContentTooShortError
from http.client import IncompleteRead
from lxml.html import fromstring, tostring

In [14]:
import re
import PyPDF2

In [4]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Functions

### Web crawling

In [9]:
# TODO: implement this with Redis or something similar
class EventQueue(object):
    
    def __init__(self, cache, header):
        self.cache = cache
        self.header = header
        self.last = None
        # Read last pushed dictionary if the cache exists
        if os.path.exists(self.cache):
            with open(self.cache, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                row = None
                for row in reader:
                    pass
                self.last = row
    
    def read(self):
        with open(self.cache, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            data = [row for row in reader]
        return(data)
    
    def push(self, rows):
        first_push = not os.path.exists(self.cache)
        with open(self.cache, 'a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, self.header) 
            if first_push:
                writer.writeheader()
            for row in rows:
                writer.writerow(row)
    

In [8]:
# Use urllib to fetch a given website
def download(url, num_retries=5, user_agent='wswp', charset='utf-8', proxy=None):
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', user_agent)
    try:
        if proxy:
            proxy_support = urllib.request.ProxyHandler({'http': proxy})
            opener = urllib.request.build_opener(proxy_support)
            urllib.request.install_opener(opener)
        resp = urllib.request.urlopen(request)
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError, IncompleteRead) as e:
        if hasattr(e, 'reason'):
            print('Download error:', e.reason)
        else:
            print('Download error: incomplete read')
        html = None
        if num_retries > 0:
            #if hasattr(e, 'code') and 500 <= e.code < 600:
            return download(url, num_retries - 1)
    return html

In [10]:
# Iterate over all published company news that are newer than a given crawl date
def crawl_frontpage(url, crawl_limit=None, crawl_last=None, wait_time=1):
    datas = deque()
    stop_crawl = False
    for page in itertools.count(0):
        html = download("{url}?page={page}".format(url=url, page=page))
        tree = fromstring(html)

        path1 = '/div[@class="c-node-teaser__top_content"]/div[@class="c-node-teaser__header"]'
        path2 = '/div[@class="c-node-teaser__publish-details"]'
        news = tree.xpath('//article')
        for div in news:
            data = {
                'title' : div.xpath('.{base}/a/h3/text()'.format(base=path1))[0],
                'href' : div.xpath('.{base}/a/@href'.format(base=path1))[0],
                'type' : div.xpath('.{base}{details}/span[contains(@class,"c-node-teaser__publish-detail--type")]/text()'.format(base=path1, details=path2))[0],
                'created' : div.xpath('.{base}{details}/span[contains(@class,"c-node-teaser__publish-detail--date")]/span/text()'.format(base=path1, details=path2))[0],
                'stocks' : ":".join(div.xpath('.{base}{details}/span[contains(@class,"c-node-teaser__publish-detail--tags")]/a/text()'.format(base=path1, details=path2))),
                'pages' : ":".join(div.xpath('.{base}{details}/span[contains(@class,"c-node-teaser__publish-detail--tags")]/a/@href'.format(base=path1, details=path2)))
            }
        
            day_created = datetime.datetime.strptime(data['created'], '%d.%m.%Y %H:%M')
            if not crawl_limit is None and day_created < crawl_limit or data == crawl_last:
                stop_crawl = True
                break
            
            datas.appendleft(data)

        if len(news) == 0 or stop_crawl:
            break

        time.sleep(wait_time)
    return(datas)


In [70]:
# Before site update
def download_report(url, download_path, num_waits=10):
    report_pdf = os.path.join(download_path, '{}.pdf'.format(url))
    if not os.path.exists(report_pdf):
        try:
            driver.get('https://inderes.fi/fi/{url}'.format(url=url))
            time.sleep(1.0)
            dl = driver.find_element(By.XPATH, '//span[@class="file"]/a') #old website
            file = dl.get_attribute('href')
            #webdriver.ActionChains(driver).move_to_element(dl).click(dl).perform()
            dl.click()
            # Rename [file].pdf into corresponding [url].pdf
            download_pdf = os.path.join(download_path, os.path.basename(file))
            while not os.path.exists(download_pdf) and num_waits > 0:
                time.sleep(0.5)
                num_waits -= 1
            os.rename(download_pdf, report_pdf)
        except Exception as e:
            print("error")

In [100]:
# After site update
def download_report_new(url, download_path, num_waits=10):
    report_pdf = os.path.join(download_path, '{}.pdf'.format(url))
    if not os.path.exists(report_pdf):
        try:
            driver.get('https://inderes.fi/research/{url}'.format(url=url))
            time.sleep(1.0)
            dl = driver.find_element(By.CSS_SELECTOR, "div[class^='ReportLink_downloadContainer'] > a")
            file = dl.get_attribute('href')
            #webdriver.ActionChains(driver).move_to_element(dl).click(dl).perform()
            dl.click()
            # Rename [file].pdf into corresponding [url].pdf
            download_pdf = os.path.join(download_path, "{}.pdf".format(os.path.basename(file)))
            while not os.path.exists(download_pdf) and num_waits > 0:
                time.sleep(0.5)
                num_waits -= 1
            os.rename(download_pdf, report_pdf)
        except Exception as e:
            print("error")

### Parsing the analysis PDF

In [13]:

# List of analysts to recognize in the file
analysts = ['Antti Viljakainen','Petri Gostowski','Juha Kinnunen','Jesse Kinnunen','Erkki Vesola',
            'Joni Grönqvist','Sauli Vilén','Sauli Vilen','Olli Koponen','Atte Riikola',
            'Olli Vilppo','Joonas Korkiakoski','Antti Luiro','Petri Kajaani',
            'Pauli Lohi','Thomas Westerholm','Matias Arola','Mikael Rautanen','Aapeli Pursimo',
            'Antti Siltanen','Rauli Juva','Petri Aho','Rasmus Skand', 'Jari Honko',
            'Anton Damstén', 'Roni Peuranheimo', 'Arttu Heikura', 'Frans-Mikael Rostedt']
# Try to match the recommendation string, print ones not found
pattern1 = re.compile(r'(RATING|SUOSITUS|RECOMMENDATION)\s*:?\s*(OSTA|LISÄÄ|VÄHENNÄ|MYY|BUY|ACCUMULATE|REDUCE|SELL)', re.IGNORECASE)
pattern2 = re.compile(r'(Osta|Lisää|Vähennä|Myy|Pidä|Accumulate|Reduce)\s*(\(aik.|Edellinen:|Previous :)\s*(Osta|Lisää|Vähennä|Myy|Pidä|Accumulate|Reduce)\)', re.IGNORECASE)
pattern3 = re.compile(r'\(aik. (Osta|Lisää|Vähennä|Myy|Pidä)\)\s*(Osta|Lisää|Vähennä|Myy|Pidä)', re.IGNORECASE)
# Read the recommendation from the found string
x1 = re.compile(r'(RATING|SUOSITUS|RECOMMENDATION)\s*:?\s*', re.IGNORECASE)
x2 = re.compile(r'\s*(\(aik.|Edellinen:|Previous :)\s*(Osta|Lisää|Vähennä|Myy|Pidä|Accumulate|Reduce)\)', re.IGNORECASE)
x3 = re.compile(r'\(aik. (Osta|Lisää|Vähennä|Myy|Pidä)\)\s*', re.IGNORECASE)

def parse_pdf(report_pdf, download_path):
    suositus = ''
    analyytikko = ''
    report = os.path.splitext(report_pdf)[0]
    report_pdf = os.path.join(download_path, report_pdf)
    # find recommendation from file contents
    try:
        reader = PyPDF2.PdfReader(report_pdf)
        for i, page in enumerate(reader.pages):
            s = page.extract_text()
            s = " ".join(s.split())
            for pattern in (pattern1, pattern2, pattern3):
                for group in pattern.finditer(s):
                    if suositus == '':
                        suositus = s[group.start():(group.end())]
            if analyytikko == '':
                analyytikko = ','.join([analyst for analyst in analysts if analyst in s])
            if i > 1:
                break
    except:
        print("error:", end="")
    # remove extras
    for x in (x1, x2, x3):
        suositus = x.sub('', suositus).lower()
    return(report, suositus, analyytikko)


In [15]:
def fix_missing_stock(data, fix_path='reports_info'):
    print('Missing stock identifier (fix: analysis_stock_missing.csv)')
    
    data['stock'].fillna('', inplace=True)
    data_stock_missing_new = data[data['stock'] == ''].copy()
    print('\tMissing before fixes: {}'.format((data_stock_missing_new['stock'] == '').sum()))

    # read fixes from file
    fn_stock_missing = os.path.join(fix_path, 'analysis_stock_missing.csv')
    if not os.path.exists(fn_stock_missing):
        data_stock_missing_new.to_csv(fn_stock_missing, index=False)
    else:
        data_stock_missing = pd.read_csv(fn_stock_missing, index_col=False, parse_dates=[2])
        data_stock_missing['stock'].fillna('', inplace=True)
        data_stock_missing_add = data_stock_missing_new[~data_stock_missing_new['url'].isin(data_stock_missing['url'])]
        data_stock_missing_new = pd.concat([data_stock_missing, data_stock_missing_add])
        data_stock_missing_new.to_csv(fn_stock_missing, index=False)

    print('\tMissing after fixes: {}'.format((data_stock_missing_new['stock'] == '').sum()))
    
    return(pd.concat([data[data['stock'] != ''], data_stock_missing_new]).sort_values('created'))

In [16]:
def fetch_names(urls):
    url_to_name = {}
    for url in urls:
        driver.get('https://www.inderes.fi/fi/{url}'.format(url=url))
        time.sleep(0.5)
        name = driver.find_element(By.XPATH, '//span[@class="c-node-teaser__publish-detail c-node-teaser__publish-detail--tags"]/a')
        url_to_name[url] = name.text
    return(url_to_name)

name_to_id = {
    'Affecto': 'affecto',
    'Ahlstrom':'ahlstrom', 
    'Ahtium': 'ahtium', 
    'Amer': 'amer-sports', 
    'Biotie Therapies': 'biotie-therapies',
    'Comptel': 'comptel', 
    'Elektrobit': 'bittium', 
    'GF Money': 'gf-money' ,
    'Hoivatilat': 'hoivatilat',
    'Huhtanen Capital': 'huhtanen-capital',
    'Lemminkäinen': 'lemminkainen',
    'Mallisalkku': 'inderes-mallisalkku',
    'Okmetic': 'okmetic', 
    'Sponda':'sponda', 
    'Taaleri': 'taaleri',
    'Technopolis': 'technopolis'    
}

def fix_missing_stock2(data, fix_path='reports_info'):
    print('Missing stock identifier fi (fix: analysis_stock_missing_name.csv) ')
    
    data_stock_missing_new2 = data.loc[data['stock'] == 'fi'].copy()
    print('\tMissing before fixes: {}'.format((data_stock_missing_new2['stock'] == 'fi').sum()))

    # read fixes from file
    fn_stock_missing = os.path.join(fix_path, 'analysis_stock_missing_name.csv')
    if not os.path.exists(fn_stock_missing):
        url_to_name = fetch_names(data_stock_missing_new2['url'])
        pd.Series(url_to_name).to_csv(fn_stock_missing, header=False)
    else:
        url_to_name = pd.read_csv(fn_stock_missing, names=['url', 'name'], header=None)
        url_to_name = dict(url_to_name.set_index('url')['name'])
        new_urls = data_stock_missing_new2.loc[~data_stock_missing_new2['url'].isin(set(url_to_name.keys())), 'url']
        url_to_name_add = fetch_names(new_urls)
        url_to_name = {**url_to_name, **url_to_name_add}
        pd.Series(url_to_name).to_csv(fn_stock_missing, header=False)

    data_stock_missing_new2['stock'] = data_stock_missing_new2['url'].map(url_to_name).map(name_to_id).fillna('')
    print('\tMissing after fixes: {}'.format((data_stock_missing_new2['stock'] == '').sum()))

    missing_names = set(url_to_name.values()).difference(set(name_to_id.keys()))
    print('\tMissing name -> stock mappings :\n\t{}'.format(missing_names))

    return(pd.concat([data[data['stock'] != 'fi'], data_stock_missing_new2]).sort_values('created'))

In [17]:
rec_map = {'osta':'osta', 'lisää':'lisää', 'vähennä':'vähennä', 'myy':'myy', 'pidä':'pidä', 
           'buy':'osta', 'accumulate':'lisää', 'reduce':'vähennä', 'sell':'myy', '-':'-'}

def fix_missing_recommendation(data, fix_path='reports_info'):
    print('Missing recommendation (fix: analysis_recommendation_missing.csv)')
    
    data['suositus'].fillna('', inplace=True)
    data_rec_missing_new = data[data['suositus'] == ''].copy()
    print('\tMissing before fixes: {}'.format((data_rec_missing_new['suositus'] == '').sum()))

    # read fixes from file
    fn_rec_missing = os.path.join(fix_path, 'analysis_recommendation_missing.csv')
    if not os.path.exists(fn_rec_missing):
        data_rec_missing_new.to_csv(fn_rec_missing, index=False)
    else:
        data_rec_missing = pd.read_csv(fn_rec_missing, index_col=False, parse_dates=[2])
        data_rec_missing['suositus'].fillna('', inplace=True)
        data_rec_missing_add = data_rec_missing_new[~data_rec_missing_new['url'].isin(data_rec_missing['url'])]
        data_rec_missing_new = pd.concat([data_rec_missing, data_rec_missing_add])
        data_rec_missing_new.to_csv(fn_rec_missing, index=False)

    print('\tMissing after fixes: {}'.format((data_rec_missing_new['suositus'] == '').sum()))
    
    data = pd.concat([data[data['suositus'] != ''], data_rec_missing_new]).sort_values('created')
    data['suositus'] = data['suositus'].map(rec_map).fillna('')

    print('\tMissing after mapping: {}'.format((data['suositus'] == '').sum()))
    
    return(data)

In [18]:
def fix_missing_analyst(data, fix_path='reports_info'):
    print('Missing analyst (fix: analysis_analyst_missing.csv)')
    
    data['analyytikko'].fillna('', inplace=True)
    data_analyst_missing_new = data[data['analyytikko'] == ''].copy()
    print('\tMissing before fixes: {}'.format((data_analyst_missing_new['analyytikko'] == '').sum()))

    # read fixes from file
    fn_analyst_missing = os.path.join(fix_path, 'analysis_analyst_missing.csv')
    if not os.path.exists(fn_analyst_missing):
        data_analyst_missing_new.to_csv(fn_analyst_missing, index=False)
    else:
        data_analyst_missing = pd.read_csv(fn_analyst_missing, index_col=False, parse_dates=[2])
        data_analyst_missing['analyytikko'].fillna('', inplace=True)
        data_analyst_missing_add = data_analyst_missing_new[~data_analyst_missing_new['url'].isin(data_analyst_missing['url'])]
        data_analyst_missing_new = pd.concat([data_analyst_missing, data_analyst_missing_add])
        data_analyst_missing_new.to_csv(fn_analyst_missing, index=False)

    print('\tMissing after fixes: {}'.format((data_analyst_missing_new['analyytikko'] == '').sum()))
    
    return(pd.concat([data[data['analyytikko'] != ''], data_analyst_missing_new]).sort_values('created'))

In [19]:
def take_random_analyst(data):
    take_random = {}
    multiple = 0
    for i, row in data.iterrows():
        analysts = row['analyytikko'].split(',')
        take_random[row['url']] = np.random.choice(analysts)
        if len(analysts) > 1:
            multiple += 1
    data['analyytikko'] = data['url'].map(take_random)
    print('\tTake random from multiple analysts {}/{}'.format(multiple, len(data)))
    return(data)

In [20]:
def fix_end_followup(data, no_update=365):
    cutoff = pd.Timestamp(datetime.datetime.now())-pd.Timedelta(no_update, 'D')
    # Get stocks last recommendation, replace by - if not updated in 365 days (end of follow-up)
    fix_last = data.groupby('stock', as_index=False)[['created', 'suositus']].last()
    fix_last.loc[fix_last['created'] < cutoff, 'suositus'] = '-'
    data = data.merge(fix_last, how='left', on=['stock', 'created'], suffixes=('', '_korjaa'))
    end_followup = (~data['suositus_korjaa'].isnull() & (data['suositus_korjaa'] != data['suositus'])).sum()
    data.loc[~data['suositus_korjaa'].isnull(), 'suositus'] = data['suositus_korjaa']
    data.drop(columns='suositus_korjaa', inplace=True)
    print('\tFix end of followup {}/{}'.format(end_followup, len(data)))
    return(data)

## Initialize selenium

In [74]:
download_path = os.path.join(os.getcwd(), 'reports')
webdriver_path = 'chromedriver.exe'

In [75]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_path,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
    "download.open_pdf_in_system_reader": False,
    "profile.default_content_settings.popups": 0}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(webdriver_path, options=chrome_options)

C:\Users\mvil4\AppData\Local\Temp\ipykernel_18232\2057988904.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_path, options=chrome_options)


## Inderes login

In [76]:
driver.get('https://classic.inderes.fi/fi')

In [77]:
login_button = driver.find_element("id", "edit-openid-connect-client-keycloak-login")
login_button.submit()

In [78]:
driver.find_element('id', 'username').send_keys(LOGIN_EMAIL)
driver.find_element('id', 'password').send_keys(LOGIN_PASSWORD + Keys.RETURN)

## Download front page updates

In [24]:
#crawl_limit = datetime.datetime.strptime('01.01.2013 00:00', '%d.%m.%Y %H:%M')

In [40]:
events = EventQueue(cache = 'frontpage.csv', 
                    header = ['title', 'href', 'type', 'created', 'stocks', 'pages'])

In [132]:
url = "https://classic.inderes.fi/fi/node/1"
datas = crawl_frontpage(url, crawl_last=events.last, crawl_limit= datetime.datetime.strptime("11.10.2023", '%d.%m.%Y'))

In [44]:
events.push(datas)

## Download reports

In [46]:
data = pd.read_csv('frontpage.csv')
# add base url and stock
data['created'] = pd.to_datetime(data['created'], format='%d.%m.%Y %H:%M', errors='coerce')
data['url'] = [url.split('/')[-1] for url in data['href']]
data['stock'] = [url.split('/')[-1] for url in data['pages'].fillna('')]
# only relevant
data = data.loc[(data['created'] >= pd.Timestamp('2013-01-01')) & (data['type'] == 'Analyysi'), ['stock', 'url', 'created']]

In [47]:
data.tail()

,stock,url,created
61547,koskisen,matala-arvostus-vaimentaa-yksittaiset-pettymykset,2023-11-20 08:36:00
61557,bbs,rahoituskierroksen-pitaisi-kantaa-ce-merkintaa...,2023-11-21 07:10:00
61559,administer,halvalla-lahtee,2023-11-21 07:59:00
61576,srv,hyva-asema-markkinan-jalleen-vetaessa,2023-11-22 08:02:00
61590,panostaja,paaomat-taytyy-saada-tuottamaan,2023-11-23 08:12:00


In [131]:
# save .pdfs to download folder
for i, url in enumerate(data['url']):
    print(url, "{}/{}".format(i+1, len(data)))
    download_report(url, download_path) # before site update
    download_report_new(url, download_path) # after site update

## Create recommendation file

### Parse reports to recommendations

In [130]:
suositukset = []
for i, pdf in enumerate(os.listdir(download_path)):
    print(pdf, "{}/{}".format(i+1, len(data)))
    suositukset.append(parse_pdf(pdf, download_path))
suositukset = pd.DataFrame(suositukset, columns=['url', 'suositus', 'analyytikko'])

In [103]:
data = data.merge(suositukset,how='left')

In [104]:
data.to_csv(os.path.join('reports_info', 'analysis_unfixed.csv'), index=False)
data.tail()

,stock,url,created,suositus,analyytikko
7036,koskisen,matala-arvostus-vaimentaa-yksittaiset-pettymykset,2023-11-20 08:36:00,lisää,Antti Viljakainen
7037,bbs,rahoituskierroksen-pitaisi-kantaa-ce-merkintaa...,2023-11-21 07:10:00,vähennä,Antti Siltanen
7038,administer,halvalla-lahtee,2023-11-21 07:59:00,osta,Juha Kinnunen
7039,srv,hyva-asema-markkinan-jalleen-vetaessa,2023-11-22 08:02:00,lisää,Olli Koponen
7040,panostaja,paaomat-taytyy-saada-tuottamaan,2023-11-23 08:12:00,lisää,Juha Kinnunen


### Fix missing stock indicator (NA/fi) and recommendation (NA)

In [115]:
data = pd.read_csv(os.path.join('reports_info', 'analysis_unfixed.csv'), parse_dates=[2])

In [116]:
data = fix_missing_stock(data)

Missing stock identifier (fix: analysis_stock_missing.csv)
	Missing before fixes: 26
	Missing after fixes: 5


In [117]:
data = fix_missing_stock2(data)

Missing stock identifier fi (fix: analysis_stock_missing_name.csv) 
	Missing before fixes: 217
	Missing after fixes: 0
	Missing name -> stock mappings :
	set()


In [118]:
data = fix_missing_recommendation(data)

Missing recommendation (fix: analysis_recommendation_missing.csv)
	Missing before fixes: 545
	Missing after fixes: 176
	Missing after mapping: 176


In [192]:
data = fix_missing_analyst(data)

Missing analyst (fix: analysis_analyst_missing.csv)
	Missing before fixes: 147
	Missing after fixes: 76


In [119]:
data = fix_missing_analyst(data)

Missing analyst (fix: analysis_analyst_missing.csv)
	Missing before fixes: 157
	Missing after fixes: 82


In [120]:
data = take_random_analyst(data)

	Take random from multiple analysts 860/7045


In [121]:
data = fix_end_followup(data)

	Fix end of followup 28/7045


In [123]:
data.loc[data['analyytikko'] == 'Sauli Vilén', 'analyytikko'] = 'Sauli Vilen'
# these were renamed
data.loc[data['stock'] == 'loudspring', 'stock'] = 'eagle-filters'
data.loc[data['stock'] == 'nightingale', 'stock'] = 'nightingale-health'
data.loc[data['stock'] == 'alexandria-group', 'stock'] = 'alexandria-pankkiiriliike'
data.loc[data['stock'] == 'sievi-capital', 'stock'] = 'kh-group'
data.loc[data['stock'] == 'soprano', 'stock'] = 'wetteri'
data.loc[data['stock'] == 'lifa-air', 'stock'] = 'pallas-air'
# Metso Minerals + Outotec = Metso Outotec fusion, Metso -> Neles renaming, Metso Outotec -> Metso renaming 
data.loc[data['stock'] == 'metso-outotec', 'stock'] = 'metso-0'
# Fellow Finance + Evli Pankki = Alisa Pankki fusion, Evli new listing
data.loc[(data['stock'] == 'fellow-pankki') & (data['created'] < pd.Timestamp('2022-04-02')), 'stock'] = 'fellow-finance' #FELLOWX
data.loc[(data['stock'] == 'fellow-pankki') & (data['created'] >= pd.Timestamp('2022-04-02')), 'stock'] = 'alisa-pankki' #ALISA
data.loc[(data['stock'] == 'evli') & (data['created'] < pd.Timestamp('2022-04-02')), 'stock'] = 'evli-pankki' #ALISA
# Reading fixes may have created some NaNs
data['stock'].fillna('', inplace=True)
data['suositus'].fillna('', inplace=True)
data['analyytikko'].fillna('', inplace=True)

### Save

In [129]:
# Save
data = data[(data['stock'] != '') & (data['suositus'] != '')]
data.to_csv('analysis.csv', index=False)